In [412]:
import pyro
import pyro.distributions as dist
from pyro.infer import Importance, EmpiricalMarginal
import matplotlib.pyplot as plt
import torch
import numpy as np
import pandas as pd
import random
from statistics import mean, stdev

## Possible steps in final report

1) Define the example problem (What will the variables be?)

2) Define the DAG(s). 

3) Code the model(s) with probabilities. 

4) Show result of an unfair classifier (gives different predictions when the protected attribute A is changed) and a fair classifier (predictions do not change when A is changed).

5) Conclusion


References: 

The paper: https://arxiv.org/pdf/1703.06856.pdf

Video on lecture by M. Kusner (author of the paper above): https://www.youtube.com/watch?v=ZfuOw02U7hs&ab_channel=TheAlanTuringInstitute

## Some definitions

A metric and algorithm to show if a given model/algorithm is fair.

Based on the video lecture model where we try to calculate unobserved confounder 'A' (law ability) for each instance to see if classifier is fair

We have the following exogenous variables that are protected:
* R = race (binary for simplicity: 0 or 1) -- this can refer to maybe "Not Race X vs. Race X"
* S = sex (binary: 0 or 1) -- 0 for female, 1 for male

and the following observed (endogenous) variables:
* G = GPA
* L = LSTAT score

which are used by the naive policy (from original nb) to predict the first year average grade (indicator of law school success here). 

* F = First year average grade

There is also an unobserved exogenous variable that we'd like to do inference on:
* A = law ability


The DAG however shows R and S are parents of G and L in a causal DAG, so by using G and L only without taking into account R and S, you have an unaware policy that is unfair (NOTE: show this by doing a counterfactual with the unaware model where you do(R=r) or do(S=s) and see a difference in F). The fair policy takes into account R and S and uses R, S, G, and L to infer what 'A' (law ability) is and use that to predict F. The paper shows a higher accuracy when doing so when using the causal DAG where A is non-deterministic of G, L, and F (vs. the other causal DAG with A1 -> G, A2 -> L, A3 -> F). 



## Original Model 

The model is defined by: https://nbviewer.jupyter.org/github/apedawi-cs/Causal-inference-discussion/blob/master/law_school.ipynb

In [337]:
def normalize(x):
    return (x - np.mean(x)) / np.std(x)

def minmax_normalizer(df):
    return (df - df.min()) / (df.max() - df.min())

def simulate_exogenous_vars(nb_obs, R_pct=0.5, S_pct=0.5):
    assert isinstance(R_pct, float) and (0 <= R_pct <= 1)
    assert isinstance(S_pct, float) and (0 <= S_pct <= 1)
    R = 1. * (np.random.uniform(low=0, high=1, size=[nb_obs, 1]) < R_pct)
    S = 1. * (np.random.uniform(low=0, high=1, size=[nb_obs, 1]) < S_pct)
    A = np.random.randn(nb_obs, 1)
    return R, S, A

def simulate_endogenous_vars(A, R, S):
    assert A.shape == R.shape == S.shape
    nb_obs = A.shape[0]
    G = A + 2.1 * R + 3.3 * S + 0.5 * np.random.randn(nb_obs, 1)
    L = A + 5.8 * R + 0.7 * S + 0.1 * np.random.randn(nb_obs, 1)
    F = A + 2.3 * R + 1.0 * S + 0.3 * np.random.randn(nb_obs, 1)
    return G, L, F

r,s,a = simulate_exogenous_vars(1000, 0.65, 0.35)
g,l,f = simulate_endogenous_vars(a,r,s)

### Model in Pyro

counterfactual tutorial: https://github.com/robertness/causalML/blob/master/tutorials/3-counterfactual/counterfactuals_in_pyro.ipynb

In [396]:
pyro.set_rng_seed(2)

exo_dist = {
    'Nr': dist.Bernoulli(torch.tensor(0.7)),
    'Ns': dist.Bernoulli(torch.tensor(0.35)),
    'Na': dist.Normal(torch.tensor(0.), torch.tensor(1.))
}

def model(exo_dist):
    # sample from bernoulli 0 or 1, 0 at 70% freq (made up)
    R = pyro.sample("R", exo_dist['Nr'])
    S = pyro.sample("S", exo_dist['Ns'])
    
    # random gaussian dist for ability 
    A = pyro.sample("A", exo_dist['Na'])
    
    
    G = pyro.sample("G", dist.Normal(A + 2.1 * R + 3.3 * S, 0.5))
    
    L = pyro.sample("L", dist.Normal(A + 5.8*R + 0.7*S, 0.1))
    
    F = pyro.sample("F", dist.Normal(A + 2.3*R + 1.*S, 0.3))

trace_handler = pyro.poutine.trace(model)
samples = pd.DataFrame(columns=['R', 'S', 'A', 'G', 'L', 'F', 'p'])
for i in range(1000):
    trace = trace_handler.get_trace(exo_dist)
    R = trace.nodes['R']['value']
    S = trace.nodes['S']['value']
    A = trace.nodes['A']['value']
    G = trace.nodes['G']['value']
    L = trace.nodes['L']['value']
    F = trace.nodes['F']['value']
    # get prob of each combination
    log_prob = trace.log_prob_sum()
    p = np.exp(log_prob)
    samples = samples.append({'R': R, 'S': S, 'A': A, 'G': G, 'L':L, 'F': F, 'p': p}, ignore_index=True)
samples.head()

,R,S,A,G,L,F,p
0,tensor(1.),tensor(0.),tensor(1.2884),tensor(3.8376),tensor(6.8743),tensor(3.4224),tensor(0.0194)
1,tensor(1.),tensor(0.),tensor(0.5731),tensor(2.9436),tensor(6.3339),tensor(2.5603),tensor(0.3029)
2,tensor(1.),tensor(0.),tensor(-0.5682),tensor(0.8972),tensor(5.2896),tensor(1.2763),tensor(0.0781)
3,tensor(1.),tensor(1.),tensor(0.6181),tensor(5.8989),tensor(7.1206),tensor(3.7707),tensor(0.2944)
4,tensor(1.),tensor(1.),tensor(-0.4774),tensor(4.0657),tensor(5.9872),tensor(2.5261),tensor(0.0490)


Can compare between using the original notebook's sample results vs. pyro version:

In [397]:
max(samples['F']), min(samples['F'])

(tensor(6.7913), tensor(-3.2315))

In [398]:
max(f), min(f)

(array([6.29013306]), array([-2.86325914]))

In [418]:
sum(samples['G'])/NUM_SAMPLES

tensor(2.5301)

In [419]:
# some means
print("G")
print(sum(samples['G'])/NUM_SAMPLES)

print("L")
print(sum(samples['L'])/NUM_SAMPLES)

print("F")
print(sum(samples['F'])/NUM_SAMPLES)

G
tensor(2.5301)
L
tensor(4.2523)
F
tensor(1.9122)


### Conditioned Model

Condition on a person being G=0, L=0, F=0, infer the exogenous variables

In [424]:
NUM_SAMPLES=1000

# exo distribution to use
exo_dist = {
    'Nr': dist.Bernoulli(torch.tensor(0.7)),
    'Ns': dist.Bernoulli(torch.tensor(0.35)),
    'Na': dist.Normal(torch.tensor(0.), torch.tensor(1.))
}

conditioned = pyro.condition(
    model,
    {'G': torch.tensor(2.5), 'L': torch.tensor(4.2), 'F': torch.tensor(1.9)}
)
post = Importance(conditioned, num_samples=NUM_SAMPLES).run(exo_dist)
R_marginal = EmpiricalMarginal(post, "R")
R_samples = [R_marginal().item() for _ in range(NUM_SAMPLES)]
R_unique, R_counts = np.unique(R_samples, return_counts=True)
print("Mean of R: ", mean(R_samples))

S_marginal = EmpiricalMarginal(post, "S")
S_samples = [S_marginal().item() for _ in range(NUM_SAMPLES)]
S_unique, S_counts = np.unique(S_samples, return_counts=True)
print("Mean of S: ", mean(S_samples))

A_marginal = EmpiricalMarginal(post, "A")
A_samples = [A_marginal().item() for _ in range(NUM_SAMPLES)]
A_unique, A_counts = np.unique(A_samples, return_counts=True)
print("Mean of A: ", mean(A_samples), "Stdev: ", stdev(A_samples))

Mean of R:  1.0
Mean of S:  1.0
Mean of A:  -2.177851146221161 Stdev:  0.06189118141212581


### Do Model

What if that same person was Male (S=1) instead?

In [425]:
# forward propagate posterior of exogenous variables:
updated_exo_dist = {
    'Nr': dist.Bernoulli(torch.tensor(mean(R_samples))),
    'Ns': dist.Bernoulli(torch.tensor(mean(S_samples))),
    'Na': dist.Normal(torch.tensor(mean(A_samples)), stdev(A_samples))
}

do_model = pyro.do(conditioned, data={"S": torch.tensor(1.)})
# use pyro.infer.Importance
do_post = pyro.infer.Importance(do_model, num_samples=NUM_SAMPLES).run(updated_exo_dist)
do_marginal = pyro.infer.EmpiricalMarginal(do_post, "A")
do_samples = [do_marginal().item() for _ in range(NUM_SAMPLES)]
do_unique, do_counts = np.unique(do_samples, return_counts=True)

In [426]:
print("Mean of A: ", mean(do_samples))

Mean of A:  -2.197156010866165


In [427]:
# difference between means:
print(mean(A_samples) - mean(do_samples))

0.019304864645004027


## Naive Policy explanation

Naive policy only takes into account G and L. It uses a combination of these 2 values and selects the top `nb_seats` instances where G+L is highest. 

In [262]:
def naive_policy(G,L,nb_seats=200):
    """Returns array of binary predictions for law school acceptance based on naive policy
    that only takes into account G and L."""
    assert G.shape == L.shape == F.shape
    nb_obs = G.shape[0]
    # P is predictions
    # set P to be same size as number of observations
    # set all to False for now 
    P = np.zeros([nb_obs,1]).astype(bool) 
    # if nb_seats is less than nb_obs, just set nb_seats to be nb_obs (everyone is accepted)
    if nb_seats > nb_obs:
        nb_seats = nb_obs 
    # get the top nb_seats values 
    ind = (normalize(G)+normalize(L)).argsort(axis=0)[-nb_seats:][::-1]
    print(ind.shape)
    # set the top nb_seats to be True by index
    P[ind] = True
    return P

## Code from Original Notebook

In [20]:
#Setting up the policies
#For better understanding check this repo
# https://github.com/mkusner/counterfactual-fairness/blob/master/law_school_classifiers.R
# Original:
# https://github.com/apedawi-cs/Causal-inference-discussion/blob/master/law_school.ipynb
# paper:
#https://arxiv.org/abs/1703.06856
# To understand the list index thing:
# https://www.kite.com/python/answers/how-to-use-numpy-argsort-in-descending-order-in-python

from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np


class NaivePolicy:
    def __init__(self):
        pass
    def evaluate(self, G, L, nb_seats=None):
        assert G.shape == L.shape
        nb_obs = G.shape[0]
        if nb_seats is None:
            nb_seats = nb_obs
        else:
            assert isinstance(nb_seats, int) and (nb_seats > 0)
            nb_seats = min(nb_obs, nb_seats)
        ind = (normalize(G) + normalize(L)).argsort(axis=0)[-nb_seats:][::-1]
        P = np.zeros([nb_obs, 1]).astype(bool)
        P[ind] = True
        return P

class UnawarePolicy:
    #Unaware = ZFYA ~ LSAT + UGPA 
    def __init__(self):
        pass
    def train(self, G, L, F):
        X_train, X_test, y_train, y_test = train_test_split(np.hstack([G, L]), F, test_size=0.33)
        self.F_reg = LinearRegression().fit(X_train,y_train)
        y_pred    = self.F_reg.predict(X_test)
        #print(np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
        
        
    def evaluate(self, G, L, nb_seats=None):
        assert G.shape == L.shape
        nb_obs = G.shape[0]
        if nb_seats is None:
            nb_seats = nb_obs
        else:
            assert isinstance(nb_seats, int) and (nb_seats > 0)
            nb_seats = min(nb_obs, nb_seats)
        F_hat = self.F_reg.predict(np.hstack([G, L]))
        ind = F_hat.argsort(axis=0)[-nb_seats:][::-1] #get the indexes in sorted ascending order using ranking
        P = np.zeros([nb_obs, 1]).astype(bool)
        P[ind] = True
        return P

class FairPolicy:
    #model-ugpa =
    #model-lsat =
    
    def __init__(self):
        pass
    def train(self, R, S, G, L):
        self.G_reg = LinearRegression().fit(np.hstack([R, S]), G)
        
        X_train, X_test, y_train, y_test = train_test_split(np.hstack([R, S]), G, test_size=0.33)
        self.G_reg = LinearRegression().fit(X_train,y_train)
        y_pred    = self.G_reg.predict(X_test)
        #print(np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
        
        self.L_reg = LinearRegression().fit(np.hstack([R, S]), L)
        
        X_train, X_test, y_train, y_test = train_test_split(np.hstack([R, S]), L, test_size=0.33)
        self.L_reg = LinearRegression().fit(X_train,y_train)
        y_pred    = self.L_reg.predict(X_test)
        #print(np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
        
        G_err = G - self.G_reg.predict(np.hstack([R, S]))
        L_err = L - self.L_reg.predict(np.hstack([R, S]))
        self.A_reg = PCA(whiten=True, n_components=1).fit(np.hstack([G_err, L_err]))
        self.sgn = np.sign(np.corrcoef(self.A_reg.transform(np.hstack([G_err, L_err])).T, G.T)[0, 1])
    def evaluate(self, R, S, G, L, nb_seats=None):
        assert R.shape == S.shape == G.shape == L.shape
        nb_obs = R.shape[0]
        if nb_seats is None:
            nb_seats = nb_obs
        else:
            assert isinstance(nb_seats, int) and (nb_seats > 0)
            nb_seats = min(nb_obs, nb_seats)
        G_err = G - self.G_reg.predict(np.hstack([R, S]))
        L_err = L - self.L_reg.predict(np.hstack([R, S]))
        A_hat = self.sgn * self.A_reg.transform(np.hstack([G_err, L_err]))
        ind = A_hat.argsort(axis=0)[-nb_seats:][::-1]
        P = np.zeros([nb_obs, 1]).astype(bool)
        P[ind] = True
        return P

In [14]:
#Utility Functions
def normalize(x):
    return (x - np.mean(x)) / np.std(x)

def minmax_normalizer(df):
    return (df - df.min()) / (df.max() - df.min())
def build_plot(P, A, R, S, G, L, F, colors=None, pc_samps=1000, figscale=6, fontsize=20):
    if colors is None:
        colors = {
            (0, 0): [(0.882, 0.529, 0.000, 1.000), (1.000, 0.647, 0.000, 0.500)],
            (1, 0): [(0.882, 0.000, 0.000, 1.000), (1.000, 0.000, 0.000, 0.500)],
            (0, 1): [(0.000, 0.882, 0.000, 1.000), (0.000, 1.000, 0.000, 0.500)],
            (1, 1): [(0.000, 0.000, 0.882, 1.000), (0.000, 0.000, 1.000, 0.500)]
        }
    gs = GridSpec(3, 4)
    gs.update(wspace=0, hspace=0)
    kwargs_hist = dict(bins=25, histtype='stepfilled', stacked=True)
    kwargs_text = dict(horizontalalignment='left', verticalalignment='top', fontsize=fontsize)
    fig = plt.figure(figsize=(4 * figscale, 3 * figscale))
    ax_dict = dict()
    for i, tup in enumerate(itertools.product([0, 1], [0, 1])):
        j, k = tup
        ind = (R == j) & (S == k)
        ax = fig.add_subplot(gs[j, k]) 
        ax.hist([A[ind & P], A[ind & ~P]], color=colors[tup], **kwargs_hist)
        ax.axvline(x=0, ls='dotted', color='black')
        ax.text(0.02, 0.98, 'R={0:}, S={1:}'.format(j, k), transform=ax.transAxes, **kwargs_text)
        ax.set_yticks([])
        ax.set_xlim([-5, 5])
        ax.set_xticks([])
        ax_dict[i] = ax
    ylim = [0, 1.05 * max([ax.get_ylim()[1] for ax in ax_dict.values()])]
    [ax.set_ylim(ylim) for ax in ax_dict.values()];
    ax = fig.add_subplot(gs[0:2, 2:])
    ax.hist([A[P], A[~P]], color=['darkgray', 'lightgray'], **kwargs_hist)
    ax.axvline(x=0, ls='dotted', color='black')
    ax.text(0.01, 0.99, 'All', transform=ax.transAxes, **kwargs_text)
    ax.set_yticks([])
    ax.set_xlim([-5, 5])
    ax.set_xticks([])
    ax = fig.add_subplot(gs[2:, 0:])
    z = ['A', 'G', 'L', 'F']
    x = range(len(z))
    df = pd.DataFrame({'A': A.flat, 'G': G.flat, 'L': L.flat, 'F': F.flat}, columns=z)
    df = minmax_normalizer(df)
    idx = np.random.choice(range(len(df)), pc_samps)
    colors = pd.DataFrame({'R': R.flat, 'S': S.flat}, columns=['R', 'S'])\
      .apply(tuple, axis=1).apply(lambda i: colors[i])
    for i in df.index[idx]:
        color = colors[i][0] if P[i] else colors[i][1]
        alpha = 0.500 if P[i] else 0.025
        ax.plot(x, df.loc[i], ls='solid', color=color, alpha=alpha)
    ax.set_ylim([0, 1])
    ax.set_xlim([x[0], x[-1]])
    ax.set_xticks(x)
    ax.set_xticklabels(z)
    [ax.axvline(x=_x, lw=1, ls='dotted', color='black') for _x in x];
    for tick in ax.yaxis.get_major_ticks():
        tick.label1On = False
        tick.tick1On = False
    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(fontsize) 
    return fig

In [15]:

import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.gridspec import GridSpec
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
nb_seats = 20000
#Main
# set up naive policy
naivePolicy = NaivePolicy()

# set up and train unaware policy
unawarePolicy = UnawarePolicy()
unawarePolicy.train(G, L, F)

# set up and train fair policy
fairPolicy = FairPolicy()
fairPolicy.train(Nr, Ns, G, L)

In [16]:
P = {
    'naive': naivePolicy.evaluate(G, L, nb_seats),
    'unaware': unawarePolicy.evaluate(G, L, nb_seats),
    'fair': fairPolicy.evaluate(S, G, L, nb_seats)
}
#build_plot(P['naive'], A, R, S, G, L, F);
#build_plot(P['unaware'], A, R, S, G, L, F);
#build_plot(P['fair'], A, R, S, G, L, F);

NameError: name 'S' is not defined